In [2]:
import pandas as pd
import mne
import numpy as np

# 1. Ruta del archivo
fname = r"/Users/lucianatarazona/Downloads/OpenBCI-RAW-2025-09-17_12-45-51 (1).txt"

# 2. Leer el archivo OpenBCI
# - comment='%' → ignora líneas que empiezan con %
# - engine='python' + sep=None → detecta si usa coma, tab o espacios
df = pd.read_csv(
    fname,
    comment='%',
    sep=None,
    engine='python'
)

# 3. A veces OpenBCI pone una fila con texto (Sample Index...). Vamos a forzar a numérico.
#    errors='coerce' convierte lo que no es número en NaN
df = df.apply(pd.to_numeric, errors='coerce')

# 4. Quitar columnas que quedaron totalmente vacías o filas que son solo NaN
df = df.dropna(axis=1, how='all')
df = df.dropna(axis=0, how='all')

# Ahora df debería ser puro número

# 5. Elegimos cuántos canales EEG queremos.
#    Supongamos que tus primeros 8 canales son EEG. Si usaste 4, cambia a range(4).
n_eeg = 8
# Si el archivo tiene menos columnas que eso, ajustamos
n_eeg = min(n_eeg, df.shape[1])

eeg_idx = list(range(n_eeg))
data = df.iloc[:, eeg_idx].to_numpy().T   # (n_channels, n_samples)

# 6. Crear nombres de canal
ch_names = [f"EEG{i+1}" for i in range(n_eeg)]

# 7. Frecuencia de muestreo (ajusta si no es 250 Hz)
sfreq = 250.0

# 8. Crear info y Raw
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
raw = mne.io.RawArray(data, info)

# 9. Recortar (si tu señal dura menos de 30 s, baja este valor)
raw.crop(tmin=0, tmax=30)

# 10. Filtrar como en tu código original
raw.load_data()
raw.filter(l_freq=1.0, h_freq=None)

# 11. Mantener EEG
raw.pick(picks="eeg", exclude="bads")

print(raw)

Creating RawArray with float64 data, n_channels=8, n_times=180227
    Range : 0 ... 180226 =      0.000 ...   720.904 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 s)

<RawArray | 8 x 7501 (30.0 s), ~479 KiB, data loaded>
